In [385]:
import os
import json
import pandas as pd
from urllib.parse import urlparse
import copy 
import ast

data_path = os.path.join("D:/", "data", "drmkc")

In [386]:
filename = "drmkc-scrape_20210218.json"

with open(os.path.join(data_path, filename), 'r') as f:
    data = json.load(f)
    f.close()

In [387]:
filename = "drr_scrape2021-03-30.json"

with open(os.path.join(data_path, filename), 'r') as f:
    data = data + json.load(f)
    f.close()

In [388]:
filename = "drr_scrape2021-04-28.json"

with open(os.path.join(data_path, filename), 'r') as f:
    data = data + json.load(f)
    f.close()

In [389]:
data_parsed = copy.deepcopy(data)

for entry in data_parsed:
    entry['links'] = [url for url in entry['links'] if "http" in url]
    #entry['links'] = list(set([urlparse(url).netloc for url in entry['links']]))
    links_parsed = []
    for url in entry['links']:
        try:
            url = urlparse(url).netloc
        except:
            url = url
        links_parsed.append(url)
    entry['links'] = list(set(links_parsed))
    entry['url'] = urlparse(entry['url']).netloc
    entry['links'] = [url for url in entry['links'] if url != entry['url']]

In [390]:
df = pd.DataFrame.from_records(data_parsed)

In [391]:
df_kw = df.explode('keywords_matched')
df_kw['keywords_matched'] = df_kw['keywords_matched'].str.strip()

In [392]:
exclude_kw = ['social media', 'government', 'participatory', 'youtube', 'nongovernmental organizations', 'notifications']
df_kw = df_kw.loc[~df_kw['keywords_matched'].isin(exclude_kw), :]

In [393]:
df_kw.head()

,url,links,date-of-access,keywords_matched
0,drmkc.jrc.ec.europa.eu,"[europa.eu, ec.europa.eu]",2021-02-18,publicly available
8,drmkc.jrc.ec.europa.eu,"[www.oecd.org, www.undrr.org, media.ifrc.org, ...",2021-02-18,agriculture
9,drmkc.jrc.ec.europa.eu,"[europa.eu, ec.europa.eu, apps.who.int, www.wh...",2021-02-18,publicly available
10,drmkc.jrc.ec.europa.eu,"[europa.eu, ec.europa.eu, effis.jrc.ec.europa....",2021-02-18,lidar
16,drmkc.jrc.ec.europa.eu,"[europa.eu, ec.europa.eu]",2021-02-18,publicly available


In [394]:
df_nw = df_kw.explode('links').loc[:, ['url', 'links']]
df_nw = df_nw.drop_duplicates().reset_index(drop = True)
df_nw = df_nw.loc[(df_nw['links'].notna()) & (df_nw['url'] != "") & (df_nw['links'] != "https:") & (df_nw['links'] != "http:"), :]

drop_url = ['twitter.com', 'm.facebook.com']
drop_links_manual = ['twitter', 'facebook', 'youtube', 'google', 'nature.com', 'zoom.us', 'goo.gl', 'bit.ly', 'flcikr', 't.co', 'medium.com', 'github', 'tandfonline', 'linkedin', 'bbc.co.uk', 'news24.com', 'vimeo', 'dx.doi.org']
drop_links = list(df_nw['links'].value_counts()[df_nw['links'].value_counts() == 1].index)

df_nw = df_nw.loc[~((df_nw['url'].isin(drop_url)) | (df_nw['links'].isin(drop_links)) | (df_nw['links'].apply(lambda link: any([drop_link in link for drop_link in drop_links_manual])))), :]

In [395]:
df_nw.shape

(85, 2)

In [396]:
df_nw.head()

,url,links
0,drmkc.jrc.ec.europa.eu,europa.eu
1,drmkc.jrc.ec.europa.eu,ec.europa.eu
2,drmkc.jrc.ec.europa.eu,www.oecd.org
4,drmkc.jrc.ec.europa.eu,media.ifrc.org
9,drmkc.jrc.ec.europa.eu,www.unwomen.org


In [397]:
for url in list(df_nw['url'].unique()):
    print(df_nw.loc[df_nw['url'] == url].shape)

(22, 2)
(1, 2)
(1, 2)
(3, 2)
(16, 2)
(38, 2)
(4, 2)


In [398]:
outname = "drr-nw_20210505.csv"

df_nw.to_csv(os.path.join(data_path, outname), index = False)

In [399]:
# New links to scrape
for entry in data:
    entry['links'] = [url for url in entry['links'] if "http" in url]
    
exclude_kw = ['social media', 'government', 'participatory', 'youtube', 'nongovernmental organizations', 'notifications']
drop_url = ['twitter.com', 'm.facebook.com']
drop_links_manual = ['twitter', 'facebook', 'youtube', 'google', 'nature.com', 'zoom.us', 'goo.gl', 'bit.ly', 'flcikr', 't.co', 'medium.com', 'github', 'tandfonline', 'linkedin', 'bbc.co.uk', 'news24.com', 'vimeo', 'dx.doi.org']
drop_domains = [url.replace("www.", "") for url in list(df_nw['url'].unique())]
drop_links = drop_links_manual + drop_domains

df = pd.DataFrame.from_records(data)
df['keywords_matched'] = df['keywords_matched'].apply(lambda kw_list: [kw.strip() for kw in kw_list])
df = df.loc[~df['keywords_matched'].apply(lambda kws: any([kw in kws for kw in exclude_kw]))]
df = df.explode('links')
df = df.dropna(subset = ['links'])

df = df.loc[~((df['url'].apply(lambda link: any([drop_link in link for drop_link in drop_url]))) | 
              (df['links'].apply(lambda link: any([drop_link in link for drop_link in drop_links])))), :].reset_index(drop = True)

In [400]:
df.head()

,url,links,date-of-access,keywords_matched
0,https://www.securityresearch-cou.eu/thethemes/...,https://ec.europa.eu/commission/index_en,2021-03-30,[situational awareness]
1,https://www.securityresearch-cou.eu/thethemes/...,https://ec.europa.eu/commission/index_en,2021-03-30,[lidar]
2,https://www.undrr.org/publications?field_publi...,https://www.unisdr.org/conference/2018/efdrr,2021-03-30,[agriculture]
3,https://www.undrr.org/publications?field_publi...,https://www.preventionweb.net/about/privacy,2021-03-30,[agriculture]
4,https://www.undrr.org/publications?field_publi...,https://www.unisdr.org/conference/2018/afrp-acdrr,2021-03-30,[agriculture]


In [401]:
start_urls = list(df['links'].unique())

outname = "drr_new-links_20210505.txt"
with open(os.path.join("..", 'materials', outname), 'w', encoding = 'utf-8') as f:
    for url in start_urls:
        f.write(url + "\n")
    f.close()

In [402]:
len(start_urls)

6538